In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 


pybullet build time: Jan 29 2025 23:19:57


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3050 Ti Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.230.02
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.230.02
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3050 Ti Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [2]:
pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity as on Earth
time_step = 1./240.
pybullet.setTimeStep(time_step) # Set the elapsed time per step

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")



In [3]:
# Load the robot
car_start_pos = [0, 0, 0.1]  # Set the initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, 0])  # Set the initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/mobile_robot_with_arm.urdf", car_start_pos, car_start_orientation)

In [4]:
import cv2
import numpy as np
import time

# Camera settings (similar to AR marker detection notebook)
fov = 60  # Field of view in degrees
image_width = 640
image_height = 480
aspect = image_width / image_height
near = 0.05  # Near clipping plane
far = 10.0   # Far clipping plane

# Compute projection matrix
projection_matrix = pybullet.computeProjectionMatrixFOV(fov, aspect, near, far)

# Set camera view for GUI
camera_distance = 2.0
camera_yaw = 45.0
camera_pitch = -30.0
camera_target_position = [0.0, 0.0, 0.5]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)


In [6]:
# Function to get link index by name
def get_link_index_by_name(robot_id, link_name):
    """
    Get the link index by searching through all joints.
    In PyBullet, child link indices are the same as joint indices.
    """
    # Search through all joints
    num_joints = pybullet.getNumJoints(robot_id)
    for i in range(num_joints):
        joint_info = pybullet.getJointInfo(robot_id, i)
        child_link_name = joint_info[12].decode('utf-8')
        if child_link_name == link_name:
            # Return the child link index (same as joint index for child links)
            return i
    
    # If not found, return None
    return None

# Get camera link indices
rgbd_camera_link_idx = get_link_index_by_name(car_id, "rgbd_camera_link")
rgbd_camera_target_link_idx = get_link_index_by_name(car_id, "rgbd_camera_target_vertual_link")

print(f"RGB-D Camera Link Index: {rgbd_camera_link_idx}")
print(f"RGB-D Camera Target Link Index: {rgbd_camera_target_link_idx}")

# If we can't find by name, we'll use a fallback method
# Actually, in PyBullet, link indices for fixed joints are the same as joint indices
# Let's try to get link state directly
if rgbd_camera_link_idx is None:
    # Alternative: search through all links
    # For now, let's assume the links exist and try to access them
    # We'll handle errors in the main loop
    print("Warning: Could not find camera link by name. Will try to use joint indices.")


RGB-D Camera Link Index: 6
RGB-D Camera Target Link Index: 7


In [ ]:
# Main loop to capture and display camera images
# Press 'q' to quit the display windows

# Create named windows for display
cv2.namedWindow('RGB Image', cv2.WINDOW_NORMAL)
cv2.namedWindow('Depth Image', cv2.WINDOW_NORMAL)
cv2.namedWindow('Segmented Image', cv2.WINDOW_NORMAL)

try:
    frame_count = 0
    while True:
        # Step simulation
        pybullet.stepSimulation()
        
        # Try to get camera link positions
        # First, let's find the correct link indices by checking all joints
        if rgbd_camera_link_idx is None or rgbd_camera_target_link_idx is None:
            # Search for camera links
            num_joints = pybullet.getNumJoints(car_id)
            for i in range(num_joints):
                joint_info = pybullet.getJointInfo(car_id, i)
                link_name = joint_info[12].decode('utf-8')
                if "rgbd_camera_link" in link_name and rgbd_camera_link_idx is None:
                    rgbd_camera_link_idx = i
                if "rgbd_camera_target" in link_name and rgbd_camera_target_link_idx is None:
                    rgbd_camera_target_link_idx = i
        
        # Get camera link positions and orientations
        try:
            if rgbd_camera_link_idx is not None:
                camera_link_state = pybullet.getLinkState(car_id, rgbd_camera_link_idx)
                camera_link_position = camera_link_state[0]  # World position
                camera_link_orientation = camera_link_state[1]  # World orientation (quaternion)
            else:
                # Fallback: use base link position + offset
                base_state = pybullet.getBasePositionAndOrientation(car_id)
                camera_link_position = [base_state[0][0] + 0.12, base_state[0][1], base_state[0][2] + 0.08]
                camera_link_orientation = base_state[1]
            
            if rgbd_camera_target_link_idx is not None:
                camera_target_state = pybullet.getLinkState(car_id, rgbd_camera_target_link_idx)
                camera_target_position = camera_target_state[0]
            else:
                # Fallback: position camera target 0.2m in front of camera
                # Convert quaternion to rotation matrix to get forward direction
                # For simplicity, assume camera points in +Y direction (forward)
                camera_target_position = [camera_link_position[0] + 0.2, 
                                         camera_link_position[1], 
                                         camera_link_position[2]]
        except:
            # If link access fails, use base position with offset
            base_state = pybullet.getBasePositionAndOrientation(car_id)
            camera_link_position = [base_state[0][0] + 0.12, base_state[0][1], base_state[0][2] + 0.08]
            camera_target_position = [camera_link_position[0] + 0.2, 
                                     camera_link_position[1], 
                                     camera_link_position[2]]
        
        # Compute view matrix
        view_matrix = pybullet.computeViewMatrix(
            cameraEyePosition=camera_link_position,
            cameraTargetPosition=camera_target_position,
            cameraUpVector=[0, 0, 1]  # Z-axis is up
        )
        
        # Get camera image (returns width, height, rgbImg, depthImg, segImg)
        width, height, rgb_img, depth_img, seg_img = pybullet.getCameraImage(
            width=image_width,
            height=image_height,
            viewMatrix=view_matrix,
            projectionMatrix=projection_matrix,
            renderer=pybullet.ER_BULLET_HARDWARE_OPENGL
        )
        
        # Process RGB image (already in correct format)
        rgb_display = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
        
        # Process depth image
        # Depth image is in meters, normalize for display (0-255)
        depth_array = np.array(depth_img)
        # Normalize depth to 0-255 range for visualization
        depth_normalized = cv2.normalize(depth_array, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        depth_colormap = cv2.applyColorMap(depth_normalized, cv2.COLORMAP_JET)
        
        # Process segmented image
        # Segmentation image contains object IDs, convert to color image
        seg_array = np.array(seg_img, dtype=np.uint8)
        seg_colormap = cv2.applyColorMap(seg_array, cv2.COLORMAP_HSV)
        
        # Display images
        cv2.imshow('RGB Image', rgb_display)
        cv2.imshow('Depth Image', depth_colormap)
        cv2.imshow('Segmented Image', seg_colormap)
        
        # Print some info every 60 frames (about 4 times per second at 240Hz)
        if frame_count % 60 == 0:
            print(f"Frame {frame_count}: RGB shape={rgb_img.shape}, Depth range=[{depth_array.min():.3f}, {depth_array.max():.3f}]m, Seg IDs={np.unique(seg_array)}")
        
        frame_count += 1
        
        # Check for 'q' key to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("Quitting camera display...")
            break
        
        # Small sleep to prevent overwhelming the system
        time.sleep(time_step)
        
except KeyboardInterrupt:
    print("Interrupted by user")
finally:
    # Clean up
    cv2.destroyAllWindows()
    print("Camera display windows closed.")
 

Frame 0: RGB shape=(480, 640, 4), Depth range=[0.844, 1.000]m, Seg IDs=[  0 255]
Frame 60: RGB shape=(480, 640, 4), Depth range=[0.844, 1.000]m, Seg IDs=[  0 255]
Frame 120: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 180: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 240: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 300: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 360: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 420: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 480: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 540: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 600: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 660: RGB shape=(480, 640, 4), Depth range=[0.842, 1.000]m, Seg IDs=[  0 255]
Frame 7

: 